## OpenFeign
OpenFeign utilizes a service contract and creates service client implementation using it. As a simplistic example:

In [ ]:
// This interface defines service contract with Github
interface GitHub {
  @RequestLine("GET /repos/{owner}/{repo}/contributors")
  List<Contributor> contributors(@Param("owner") String owner, @Param("repo") String repo);
}

class Contributer {
  String login;
  int contributions;
}

public class Demo {
  public static void main(String... args) {
    GitHub github = Feign.builder()
                         .decoder(new GsonDecoder())
                         .target(GitHub.class, "https://api.github.com");

    // Fetch and print a list of the contributors to this library.
    List<Contributor> contributors = github.contributors("OpenFeign", "feign");
    for (Contributor contributor : contributors) {
      System.out.println(contributor.login + " (" + contributor.contributions + ")");
    }
  }
}

### Contract
`Contract` defines the contract between the interface and how the underlying client should work. The default `Contract` bundled with Feign utilises annotations like `@RequestLine`, `@Param`, `@Headers`, `@QueryMap`, `@HeaderMap` and `@Body`. Sample interface demonstrating various options:

In [ ]:
interface MyApi {
  @RequestLine("GET /api/documents/{contentType}")
  @Headers("Accept: {contentType}")
  String getDocumentByType(@Param("contentType") String type);
    
  @RequestLine("POST /")
  void postData(@HeaderMap Map<String, Object> headerMap);
  
  @RequestLine("GET /find")
  Book findBook(@QueryMap Map<String, Object> queryMap);
    
  @RequestLine("POST /LOGIN")
  @Headers("Content-Type: application/json")
  // json curly braces must be escaped!
  @Body("%7B\"user_name\": \"{user_name}\", \"password\": \"{password}\"%7D")
  void LOGIN(@Param("user_name") String user, @Param("password") String password);
}

Users using JAX-RS would prefer using JAX-RS conforming service contract. In that case, we need to specify contract while building the Feign client:

In [ ]:
interface GitHub {
  @GET @Path("/repos/{owner}/{repo}/contributors")
  List<Contributor> contributors(@PathParam("owner") String owner, @PathParam("repo") String repo);
}

public class Example {
  public static void main(String[] args) {
    GitHub github = Feign.builder()
                       .contract(new JAXRSContract())
                       .target(GitHub.class, "https://api.github.com");
  }
}

A `Contract` implementation defines what annotations and values are valid on interfaces. It is called to parse the methods in the class that are linked to HTTP requests.

In [ ]:
public interface Contract {
    // This method is called to parse the methods in the class that are linked to HTTP requests.
    // A typical implementation would iterate over methods of the supplied targer type
    List<MethodMetadata> parseAndValidateMetadata(Class<?> targetType);
    
    //...
}

`MethodMetadata` contains all the necessary information to form a proper URL. Other available `Contract` implementations are `JAXRS2Contract` and `JAXRS3Contract`. Spring would provide its own contract implementation.

`MethodMetadata` is eventually used to construct `Request` object used by `Client`.

### Encoder
Encodes an object into an HTTP request body. By configuring an Encoder, we can send a type-safe request body. 

In [ ]:
// instead of a String representing request body
interface LoginClient {
  @RequestLine("POST /")
  @Headers("Content-Type: application/json")
  void login(String content);
}

// we can have a Java class representing the same
interface LoginClient {
  @RequestLine("POST /")
  @Headers("Content-Type: application/json")
  void login(LoginCredentials credentials);
}

// JAX-RS
interface LoginClient {
  @POST
  @Consumes("application/json")
  void login(LoginCredentials credentials);
}

Some examples include `SpringEncoder`, `SpringFormEncoder`, `JacksonEncoder`.

In [ ]:
public interface Encoder {
  void encode(Object object, Type bodyType, RequestTemplate template) throws EncodeException;
}

`JacksonEncoder` for example, encodes in this manner:

In [ ]:
@Override
public void encode(Object object, Type bodyType, RequestTemplate template) {
    try {
      JavaType javaType = mapper.getTypeFactory().constructType(bodyType);
      template.body(mapper.writerFor(javaType).writeValueAsBytes(object), Util.UTF_8);
    } catch (JsonProcessingException e) {
      throw new EncodeException(e.getMessage(), e);
    }
}

### Client
Feign lets us specify the HTTP client to use such as *OkHttpClient* or *Apache HTTP Client*. Any client that we intend to use must conform to:

In [ ]:
// Option class contains value for connect timeout, read timeout and
// whether to follow redirects or not
public interface Client {
  Response execute(Request request, Options options) throws IOException;
}

As an example, if we decide to use Apache HTTPClient 5, it would look like:

In [ ]:
public final class ApacheHttp5Client implements Client {
  private final HttpClient client;

  @Override
  public Response execute(Request request, Request.Options options) throws IOException {
    ClassicHttpRequest httpUriRequest;
    try {
      httpUriRequest = toClassicHttpRequest(request, options);
    } catch (final URISyntaxException e) {
      throw new IOException("URL '" + request.url() + "' couldn't be parsed into a URI", e);
    }
    final HttpHost target = HttpHost.create(URI.create(request.url()));
    final HttpClientContext context = configureTimeouts(options);

    final ClassicHttpResponse httpResponse =
        (ClassicHttpResponse) client.execute(target, httpUriRequest, context);
    return toFeignResponse(httpResponse, request);
  }
  
    
  // Other methods  
}

For Apache HTTP Client 5 to be TLS aware, we need to set the `javax.net.ssl.*` properties.

The job of the client is to covert feign `Request` object into the http client specific request object; execute the request; and finally convert http client specific response object into feign `Response` object.

### Decoder
Decodes an HTTP response into a single object of the given type. Invoked when `Response.status()` is in the 2xx range and the return type is neither void nor `Response`.

In [ ]:
import java.io.IOException;
import java.lang.reflect.Type;
import feign.FeignException;
import feign.Response;

public interface Decoder {
  Object decode(Response response, Type type) throws IOException, DecodeException, FeignException;
}

As an example, `GsonDecoder` works on JSON outputs:

In [ ]:
GitHub github = Feign.builder()
                     .decoder(new GsonDecoder())
                     .target(GitHub.class, "https://api.github.com");

User using Jackson can use `JacksonDecoder`:

In [ ]:
Github github = Feign.builder()
                .contract(new JAXRS3Contract())
                .decoder(new JacksonDecoder())
                .target(Github.class, "https://api.github.com");

An example `Decoder` implementation for JAX-RS based contract would look like:

In [ ]:
public class JAXRSDecoder implements Decoder {
    private Decoder delegate;

    public JAXRSDecoder(Decoder delegate) {
        this.delegate = delegate;
    }

    @Override
    public Object decode(Response response, Type type) throws IOException {
        if (jakarta.ws.rs.core.Response.class.equals(type)) {
            return decode(response);
        }

        return delegate.decode(response, type);
    }

    private Object decode(Response response) throws IOException {
        String bodyStr = IOUtils.toString(response.body().asInputStream(), Charsets.UTF_8);

        jakarta.ws.rs.core.Response.ResponseBuilder responseBuilder = jakarta.ws.rs.core.Response
                .status(response.status())
                .entity(bodyStr);
        for (Map.Entry<String, Collection<String>> entry : response.headers().entrySet()) {
            responseBuilder.header(entry.getKey(), entry.getValue());
        }

        return responseBuilder.build();
    }
}

### Options
Lets you specify additional configuration related to the client. It is upto the client to acknowledge and interpret the options.

In [ ]:
Bank bank = Feign.builder()
        .decoder(new AccountDecoder())
        // Connect timeout, Read timeout and Follow Redirect
        .options(new Request.Options(10, TimeUnit.SECONDS, 60, TimeUnit.SECONDS, true))
        .target(Bank.class, "https://api.examplebank.com");

### Request Interceptor
When we need to change all requests, regardless of their target, we can configure a `RequestInterceptor`.

In [ ]:
Bank bank = Feign.builder()
             .decoder(accountDecoder)
             .requestInterceptor(new BasicAuthRequestInterceptor(username, password))
             .target(Bank.class, "https://api.examplebank.com");

In [ ]:
public interface RequestInterceptor {
  void apply(RequestTemplate template);
}

### ErrorDecoder
Responses where Response.status() is not in the 2xx range are classified as errors, addressed by the `ErrorDecoder`. If we need more control over handling unexpected responses, Feign instances can register a custom `ErrorDecoder` via the builder. All responses that result in an HTTP status not in the 2xx range will trigger the `ErrorDecoder`'s `decode` method. If we want to retry the request again, we can throw a `RetryableException`.

In [ ]:
public class MyErrorDecoder implements ErrorDecoder {
    @Override
    public Exception decode(String methodKey, Response response) {
        return new WebApplicationException(new FeignJaxRsResponse);
    }
}

### Retryer
Feign, by default, will automatically retry `IOExceptions`, regardless of HTTP method, treating them as transient network related exceptions, and any `RetryableException` thrown from an `ErrorDecoder`. To customize this behavior, register a custom `Retryer` instance via the builder.

In [ ]:
public interface Retryer extends Cloneable {
  // if retry is permitted, return (possibly after sleeping). Otherwise propagate the exception
  void continueOrPropagate(RetryableException e);
    
  Retryer clone();
}

Below is an example `Retryer`:

In [ ]:
public class CustomRetryer implements Retryer {
    private final int MAX_ATTEMPT;
    private final long RETRY_INTERVAL;
    
    private int retryAttempt;
    
    public CustomRetryer(int maxAttempt, long retryInterval) {
        MAX_ATTEMPT = maxAttempt;
        RETRY_INTERVAL = retryInterval;
    }
    
    @Override
    public void continueOrPropagate(RetryableException e) {
        if(++attempt > MAX_ATTEMPT) {
            throw e;
        }
        
        try {
            Thread.sleep(retryInterval);
        } catch (InterruptedException ignored) {
            Thread.currentThread().interrupt();
        }
    }
    
    // Important to have clone method since Feign uses this to make copy of
    // Retryer to use for each invocation
    @Override
    public Retryer clone() {
        return new CustomRetryer(MAX_ATTEMPT, RETRY_INTERVAL);
    }
}

A `Retryer` instance will be created for each `Client` execution, allowing us to maintain state bewteen each request if desired. The `RetryeableException` class also contains a reference to the `Request` object, allowing us to modify the request for the next retry attempt. It also contains the response body and headers.

## Request Response Flow
<img src="images/openfeign.png" />

## Spring Cloud OpenFeign
Lets us configure Feign clients using annotations. Example:

In [ ]:
@SpringBootApplication
@EnableFeignClients
public class AppMain {
    public static void main(String[] args) {
        SpringApplication.run(Application.class, args);
    }
}

The `EnableFeignClients` annotation ensures that the current package is scanned for interfaces having `FeignClient` annotation. We can specify the base package to be scanned using `basePackages` or `basePackageClasses`. There is also a way to specify elements of clients (like decoder, encoder, etc) to be common for all clients by passing a class containing `@Bean` methods:

In [ ]:
@EnableFeignClients(defaultConfiguration = {DefaultFeignClientConfiguration.class})

`FeignClientConfiguration` is the default source of Feign components for all clients in case `defaultCofiguration` is not provided.

In [ ]:
// Contains default encoder, decoder, contract, etc
@Configuration(proxyBeanMethods = false)
public class FeignClientsConfiguration {
    @Bean
    @ConditionalOnMissingBean
    public Decoder feignDecoder(ObjectProvider<HttpMessageConverterCustomizer> customizers) {
        return new OptionalDecoder(new ResponseEntityDecoder(new SpringDecoder(messageConverters, customizers)));
    }

    // SpringEncoder is the default encoder. It sets null charset for binary content types and UTF-8 for all the other ones. 
    // We can modify this behaviour to derive the charset from the Content-Type header charset instead by setting the value 
    // of feign.encoder.charset-from-content-type to true.
    @Bean
    @ConditionalOnMissingBean
    @ConditionalOnMissingClass("org.springframework.data.domain.Pageable")
    public Encoder feignEncoder(ObjectProvider<AbstractFormWriter> formWriterProvider,
            ObjectProvider<HttpMessageConverterCustomizer> customizers) {
        return springEncoder(formWriterProvider, encoderProperties, customizers);
    }
    
    @Bean
    @ConditionalOnMissingBean
    public Contract feignContract(ConversionService feignConversionService) {
        boolean decodeSlash = feignClientProperties == null || feignClientProperties.isDecodeSlash();
        return new SpringMvcContract(parameterProcessors, feignConversionService, decodeSlash);
    }
    
    // Default retry behaviour is no retry. This retrying behavior is different from the Feign default one, 
    // where it will automatically retry IOExceptions, treating them as transient network related exceptions, 
    // and any RetryableException thrown from an ErrorDecoder.
    @Bean
    @ConditionalOnMissingBean
    public Retryer feignRetryer() {
        return Retryer.NEVER_RETRY;
    }
    
    // ... other beans
}

Individual Feign clients are declaratively created through `FeignClient` annotation. Usually we provide `name` and `url` for each Feign client. However, if `url` is missing and Spring Cloud LoadBalancer is present, a load balanced version of the client is created and the name is assumed to be the service id.

In [ ]:
@FeignClient("github")  // this name is used to create Spring Cloud Load Balancer client 
@Path("/repos/{owner}/{repo}/contributors")
interface GitHub {
  @GET
  Response contributors(@PathParam("owner") String owner, @PathParam("repo") String repo);
}

We can configure specific configuration per client by (`FooConfiguration` beans override `FeignClientsConfiguration`):

In [ ]:
// Do not add @Configuration to FooConfiguration else beans defined here will become the default bean
@FeignClient(name = "stores", configuration = FooConfiguration.class)
public interface StoreClient {

There is also a provision to override behaviour (and components) of individual clients through properties defined in *application.yml*:
```yml
spring:
    cloud:
        openfeign:
            client:
                config:
                    stores: # Replace this with default to affect all the clients
                        connectTimeout: 5000
```

Older versions had `feign.client` as prefix. All configurations listed [here](https://docs.spring.io/spring-cloud-openfeign/reference/configprops.html)

We can also achieve something similar through configuration properties:
```yml
spring:
    cloud:
        openfeign:
            client:
                config:
                    default:
                        connectTimeout: 5000
                        readTimeout: 5000
```

In older version of SC OpenFeign, the prefix used to be `feign.client`.  

Individual feign clients are declaratively created through `FeignClient` annotation. Usually we provide `name` and `url` for each feign client. However, if `url` is missing and Spring Cloud LoadBalancer is present, a load balanced version of the client is created and the name is assumed to be the service id.

In [ ]:
@FeignClient("github")  // this name is used to create Spring Cloud Load Balancer client 
@Path("/repos/{owner}/{repo}/contributors")
interface GitHub {
  @GET
  Response contributors(@PathParam("owner") String owner, @PathParam("repo") String repo);
}

Every `FeignClient` component also contains components from `FeignClientsConfiguration` class. This class contains configuration for Feign components like `Decoder`, `Encoder`, `Contract`, etc. We can configure specific configuration per client by (`FooConfiguration` beans override `FeignClientsConfiguration`):

In [ ]:
// Do not add @Configuration else beans defined here will become the default bean
@FeignClient(name = "stores", configuration = FooConfiguration.class)
public interface StoreClient {

As mentioned earlier, host URL, can be specified by setting the url property. In case we want service discovery to provide that, we need to not pass in this property.

In [ ]:
@FeignClient(name = "stores", url = "http://stores.com")
public interface StoreClient {

There may be instance where we need two different clients that point to the same URL. In this case just specifying the name would not be enough. Therefore, we would need to pass in `contextId` which would be used (instead of name) to uniquely identify a client.

In [ ]:
@FeignClient(name = "stores", contextId="firstStore")
public interface StoreClient {

We can also configure Feign clients through properties, for example in `application.yaml`
```yaml
spring:
    cloud:
        openfeign:
            client:
                config:
                    stores:
                        connectTimeout: 5000
                        readTimeout: 5000
                        loggerLevel: full
                        contract: com.example.JAXRS3Contract
```

### Decoder
Spring sets `ResponseEntityDecoder` as the default `Encoder`

```yaml
feign:
    client:
        config:
            default:
                encoder: com.example.SimpleEncoder
                decoder: com.example.SimpleDecoder
```

### Client
If Spring Cloud LoadBalancer is on the classpath, `FeignBlockingLoadBalancerClient` is used. If none of them is on the classpath, the default feign client is used. If we have Spring Retry in the classpath and `spring.cloud.loadbalancer.retry.enabled` is set to true, `RetryableFeignBlockingLoadBalancerClient` is used instead. Both the implementations make use of service discovery and make load balanced service calls.

Both the above implementations delegate to an actual HTTP Client such as `OkHttpClient` or `ApacheHttpClient` or `ApacheHC5` based on the following properties:
- `feign.okhttp.enabled`
- `feign.httpclient.enabled`
- `feign.httpclient.hc5.enabled`

### ErrorDecoder
Spring does not provider a default bean for `ErrorDecoder`. 

```yaml
feign:
    client:
        config:
            default:
                errorDecoder: com.example.SimpleErrorDecoder
```